In [3]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df=pd.read_excel("C:\\Users\\Dimpalbabu\\Desktop\\Dataset\\movie_dataset.xlsx", index_col=0)
df=df.fillna(0)
df

,action 1,action 2,action 3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2,1.0
user 2,5.0,3.0,3.0,2.0,2,0.0
user 3,1.0,0.0,0.0,4.0,5,4.0
user 4,0.0,2.0,1.0,0.0,4,3.0
user 5,1.0,0.0,2.0,3.0,3,4.0


In [5]:
def standardize(row):
    new_row = (row - row.mean())/(row.max() - row.min())
    return new_row

ratings_std = df.apply(standardize)
item_similarity = cosine_similarity(ratings_std.T) 
item_similarity

    
    

array([[ 1.        ,  0.70668875,  0.81368151, -0.18049872, -0.81368151,
        -0.91410609],
       [ 0.70668875,  1.        ,  0.72310153, -0.79056942, -0.72310153,
        -0.84337386],
       [ 0.81368151,  0.72310153,  1.        , -0.45018379, -1.        ,
        -0.80218063],
       [-0.18049872, -0.79056942, -0.45018379,  1.        ,  0.45018379,
         0.49236596],
       [-0.81368151, -0.72310153, -1.        ,  0.45018379,  1.        ,
         0.80218063],
       [-0.91410609, -0.84337386, -0.80218063,  0.49236596,  0.80218063,
         1.        ]])

In [6]:
item_similarity_df = pd.DataFrame(item_similarity, index = df.columns,columns=df.columns)
item_similarity_df

,action 1,action 2,action 3,romantic1,romantic2,romantic3
action 1,1.000000,0.706689,0.813682,-0.180499,-0.813682,-0.914106
action 2,0.706689,1.000000,0.723102,-0.790569,-0.723102,-0.843374
action 3,0.813682,0.723102,1.000000,-0.450184,-1.000000,-0.802181
romantic1,-0.180499,-0.790569,-0.450184,1.000000,0.450184,0.492366
romantic2,-0.813682,-0.723102,-1.000000,0.450184,1.000000,0.802181
romantic3,-0.914106,-0.843374,-0.802181,0.492366,0.802181,1.000000


In [8]:
# ltes make recommendation

def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name] * (user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

get_similar_movies("romantic3",3)
    

romantic3    0.500000
romantic2    0.401090
romantic1    0.246183
action 3    -0.401090
action 2    -0.421687
action 1    -0.457053
Name: romantic3, dtype: float64

In [15]:
# for a dummy user
action_lover = [("action 1",5),("romantic1",1),("romantic3",2)]

similar_movies = pd.DataFrame()

for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating), ignore_index = True)
    
similar_movies.sum().sort_values(ascending = False)


action 2     3.374263
action 1     3.227801
action 3     3.110570
romantic1   -2.197430
romantic2   -3.110570
romantic3   -3.523814
dtype: float64

In [ ]:
# for a real movie dataset


In [21]:
ratings = pd.read_csv("C:\\Users\\Dimpalbabu\\Desktop\\Dataset\\movie_ratings.csv", index_col = 0)
ratings = ratings.drop(['year'], axis=1)
ratings.head()

,movie,rating,user_id
0,Gladiator,8.5,1187559
1,Memento,8.5,1014199
2,Snatch,8.3,707162
3,Requiem for a Dream,8.3,688394
4,X-Men,7.4,526411


In [30]:
user_ratings = ratings.pivot_table(index = ['user_id'], columns = ['movie'], values = 'rating')
user_ratings.head()

movie,Game of Thrones,(500) Days of Summer,10 Cloverfield Lane,12 Years a Slave,127 Hours,13 Reasons Why,1408,17 Again,2 Fast 2 Furious,2012,...,Yes Man,Ying xiong,Yip Man,You Don't Mess with the Zohan,Zero Dark Thirty,Zodiac,Zombieland,Zoolander,Zootopia,xXx
user_id,,,,,,,,,,,,,,,,,,,,,
93428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
user_ratings = user_ratings.fillna(0)
user_ratings.head()

movie,Game of Thrones,(500) Days of Summer,10 Cloverfield Lane,12 Years a Slave,127 Hours,13 Reasons Why,1408,17 Again,2 Fast 2 Furious,2012,...,Yes Man,Ying xiong,Yip Man,You Don't Mess with the Zohan,Zero Dark Thirty,Zodiac,Zombieland,Zoolander,Zootopia,xXx
user_id,,,,,,,,,,,,,,,,,,,,,
93428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# building similarity matrix using pearson mothed

In [33]:
item_similarity_df = user_ratings.corr(method = 'pearson')
item_similarity_df.head(50)

movie,Game of Thrones,(500) Days of Summer,10 Cloverfield Lane,12 Years a Slave,127 Hours,13 Reasons Why,1408,17 Again,2 Fast 2 Furious,2012,...,Yes Man,Ying xiong,Yip Man,You Don't Mess with the Zohan,Zero Dark Thirty,Zodiac,Zombieland,Zoolander,Zootopia,xXx
movie,,,,,,,,,,,,,,,,,,,,,
Game of Thrones,1.000000,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
(500) Days of Summer,-0.001114,1.000000,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
10 Cloverfield Lane,-0.001114,-0.001114,1.000000,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
12 Years a Slave,-0.001114,-0.001114,-0.001114,1.000000,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
127 Hours,-0.001114,-0.001114,-0.001114,-0.001114,1.000000,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
13 Reasons Why,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,1.000000,-0.001114,-0.001114,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
1408,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,1.000000,-0.001114,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
17 Again,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,1.000000,-0.001114,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114
2 Fast 2 Furious,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,1.000000,-0.001114,...,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114,-0.001114


In [ ]:
# ltes make recommendation

def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name] * (user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [34]:
# for a dummy user
action_lover = [("Game of Thrones",5),("(500) Days of Summer",1),("127 Hours",2)]

similar_movies = pd.DataFrame()

for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating), ignore_index = True)
    
similar_movies.sum().sort_values(ascending = False)

Game of Thrones                                   2.502227
Mindhunter                                       -0.000557
The VVitch: A New-England Folktale               -0.000557
King Arthur: Legend of the Sword                 -0.000557
From Hell                                        -0.000557
The Score                                        -0.000557
Unstoppable                                      -0.000557
The Family Man                                   -0.000557
Mr. Deeds                                        -0.000557
The Hitchhiker's Guide to the Galaxy             -0.000557
It Follows                                       -0.000557
Community                                        -0.000557
A Series of Unfortunate Events                   -0.000557
Whiplash                                         -0.000557
Freaky Friday                                    -0.000557
Breaking Bad                                     -0.000557
The Pianist                                      -0.0005